In [22]:
# Import dependencies

import pandas as pd
import gmaps
import requests

In [23]:
# Import the Google Map API Key

from config import g_key

In [24]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Import the CSV file from the part 1 of the challenge

city_weather_df=pd.read_csv("challenge_data/WeatherPy_database.csv")
city_weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(Inches),Snow(Inches)
0,0,Arraial Do Cabo,BR,-22.97,-42.03,73.17,86,0,10.36,clear sky,0.0,0
1,1,San Patricio,MX,19.22,-104.70,78.62,70,77,1.86,broken clouds,0.0,0
2,2,Vaini,TO,-21.20,-175.20,77.00,88,75,4.70,broken clouds,0.0,0
3,3,Mar Del Plata,AR,-38.00,-57.56,53.01,93,100,17.34,overcast clouds,0.0,0
4,4,Fort Dix,US,40.03,-74.62,68.00,100,1,3.20,clear sky,0.0,0


In [15]:
# Ask the customer to specify for the preference of rain, snow, max and min temperature.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_preference=input("Do you want it to be raining? (yes/no)")
snow_preference=input("Do you want it to be snowing? (yes/no)")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no)yes
Do you want it to be snowing? (yes/no)no


In [19]:
# Filter the dataset to find the cities that fit the criteria.
if rain_preference=="yes" and snow_preference=="yes":

    preferred_cities_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= max_temp) &
                                              (city_weather_df["Max Temp"] >= min_temp) &
                                              (city_weather_df["Rain(Inches)"]>0) & 
                                              (city_weather_df["Snow(Inches)"]>0)]
elif rain_preference=="yes" and snow_preference=="no":
    preferred_cities_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= max_temp) &
                                              (city_weather_df["Max Temp"] >= min_temp) &
                                              (city_weather_df["Rain(Inches)"]>0) & 
                                              (city_weather_df["Snow(Inches)"]==0)]

elif rain_preference=="no" and snow_preference=="yes":
    preferred_cities_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= max_temp) &
                                              (city_weather_df["Max Temp"] >= min_temp) &
                                              (city_weather_df["Rain(Inches)"]==0) & 
                                              (city_weather_df["Snow(Inches)"]>0)]
elif rain_preference=="no" and snow_preference=="no":
    preferred_cities_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= max_temp) &
                                              (city_weather_df["Max Temp"] >= min_temp) &
                                              (city_weather_df["Rain(Inches)"]==0) & 
                                              (city_weather_df["Snow(Inches)"]==0)] 
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(Inches),Snow(Inches)
25,25,Bijie,CN,27.31,105.29,75.72,78,100,5.08,moderate rain,1.22,0
27,27,Alofi,NU,-19.06,-169.92,77.00,78,97,11.41,light rain,0.15,0
51,51,Roxas,PH,11.59,122.75,86.63,73,84,9.55,moderate rain,1.47,0
59,59,Faanui,PF,-16.48,-151.75,78.39,85,100,18.95,light rain,0.29,0
145,145,Lorengau,PG,-2.02,147.27,84.47,70,25,0.96,light rain,0.83,0


In [25]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
25,Bijie,CN,75.72,moderate rain,27.31,105.29,
27,Alofi,NU,77.00,light rain,-19.06,-169.92,
51,Roxas,PH,86.63,moderate rain,11.59,122.75,
59,Faanui,PF,78.39,light rain,-16.48,-151.75,
145,Lorengau,PG,84.47,light rain,-2.02,147.27,
174,Motygino,RU,76.91,light rain,58.18,94.76,
177,Ixtapa,MX,77.11,light rain,20.70,-105.20,
182,Bacolod,PH,88.11,moderate rain,10.67,122.95,
228,Tabou,CI,76.35,light rain,4.42,-7.35,
237,Road Town,VG,82.99,light rain,18.42,-64.62,


In [26]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [27]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.


In [28]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
25,Bijie,CN,75.72,moderate rain,27.31,105.29,Four Points by Sheraton Bijie
27,Alofi,NU,77.00,light rain,-19.06,-169.92,Taloa Heights
51,Roxas,PH,86.63,moderate rain,11.59,122.75,Urban Manor
59,Faanui,PF,78.39,light rain,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
145,Lorengau,PG,84.47,light rain,-2.02,147.27,Lorengau Harbourside Hotel
174,Motygino,RU,76.91,light rain,58.18,94.76,"Gostinitsa ""Uyut"""
177,Ixtapa,MX,77.11,light rain,20.70,-105.20,Container Inn Puerto Vallarta
182,Bacolod,PH,88.11,moderate rain,10.67,122.95,Planta Hotel & Residences
228,Tabou,CI,76.35,light rain,4.42,-7.35,hotêl doufoulougou
237,Road Town,VG,82.99,light rain,18.42,-64.62,"ZINGARA 76ft Catamaran, full crew-all included"


In [43]:
# Set up the pop-up box for each location on the gmap.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a map for the vacation spots and marker for each city.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
# Create the output file (CSV).
output_data_file = "challenge_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")
